In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

In [3]:
### Access the New York Times API

In [72]:
# Set the base URL
nyt_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"


# Build URL parameters
nyt_params = {
    'fq': filter_query,
    'sort': sort,
    'fl': field_list,
    'begin_date': begin_date,
    'end_date': end_date,
    'api-key': nyt_api_key
}

# Function to build the query URL
def build_query_url(base_url, params):
    query_params = "&".join([f"{key}={value}" for key, value in params.items()])
    return base_url + query_params


# Print the first 5 rows of the reviews_list
if reviews_list:
    print("Preview of New York Times Data:")
    print(json.dumps(reviews_list[:5], indent=4))
else:
    print("No reviews were retrieved.")


# Convert reviews_list to a Pandas DataFrame using json_normalize()
if reviews_list:
    reviews_df = pd.json_normalize(reviews_list)
    reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
else:
    print("No reviews were retrieved.")

# Print the first 5 rows of the normalized DataFrame
print("Normalized DataFrame (first 5 rows):")
print(reviews_df.head())

# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df['title'].dropna().to_list()

# Preview the titles in a readable format
print("Preview of Titles List:")
for title in titles[:20]:  # Print the first 20 titles for readability
    print(f"'{title}'")

# Build the query URL
query_url = build_query_url(nyt_url, nyt_params)
print(f"Query URL: {query_url}")

Preview of New York Times Data:
[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
    

In [5]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19

    # create query with a page number
    # API results show 10 articles at a time
    # Make a "GET" request and retrieve the JSON
    # Add a twelve second interval between queries to stay within API query limits
for page in range(20):
    nyt_params['page'] = page
    query_params = "&".join([f"{key}={value}" for key, value in nyt_params.items()])
    query_url = nyt_url + query_params
    response = requests.get(query_url)
    time.sleep(12)
# Try and save the reviews to the reviews_list
 # loop through the reviews["response"]["docs"] and append each review to the list
 # Print the page that was just retrieved
# Print the page number that had no results then break from the loop
    
    try:
        reviews = response.json()
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        print(f"Successfully retrieved page {page}")
    except Exception as e:
        print(f"No results on page {page}, exiting loop")
        print(f"Error: {e}")
        break

Successfully retrieved page 0
Successfully retrieved page 1
Successfully retrieved page 2
Successfully retrieved page 3
Successfully retrieved page 4
Successfully retrieved page 5
Successfully retrieved page 6
Successfully retrieved page 7
Successfully retrieved page 8
Successfully retrieved page 9
Successfully retrieved page 10
Successfully retrieved page 11
Successfully retrieved page 12
Successfully retrieved page 13
Successfully retrieved page 14
Successfully retrieved page 15
Successfully retrieved page 16
Successfully retrieved page 17
Successfully retrieved page 18
Successfully retrieved page 19


In [73]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [74]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)

In [75]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [76]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df['title'].dropna().to_list()
print("Preview of Titles List:")
for title in titles[:10]:  # Print the first 20 titles for readability
    print(f"'{title}'")


Preview of Titles List:
'The Attachment Diaries'
'What’s Love Got to Do With It?’ Probably a Lo'
'You Can Live Forever'
'A Tourist’s Guide to Love'
'Other People’s Children'
'One True Loves'
'The Lost Weekend: A Love Story'
'A Thousand and One'
'Your Place or Mine'
'Love in the Time of Fentanyl'


In [77]:
### Access The Movie Database API

In [78]:
# Prepare The Movie Database query
# Directly set the TMDb API key string
# Set the TMDb API key and base URLs
tmdb_search_url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_movie_url = "https://api.themoviedb.org/3/movie/"
tmdb_key_string = "&api_key=" + tmdb_api_key

In [79]:
# Create an empty list to store the TMDb results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 1
# Loop through the titles
# Check if we need to sleep before making a request    
titles = reviews_df['title'].dropna().to_list()

for title in titles:
    if request_counter % 50 == 0:
        print("Sleeping for 60 seconds to avoid hitting rate limits...")
        time.sleep(60)
        
# Perform a "GET" request for The Movie Database    
# Build the search query URL
    search_query_url = f"{tmdb_search_url}{title}{tmdb_key_string}"
    search_response = requests.get(search_query_url)

   # Add 1 to the request counter
    
    request_counter += 1


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    
    try:
        search_data = search_response.json()
        if not search_data['results']:
            print(f"No movie found for title: {title}")
            continue
        
        # Get movie id
        movie_id = search_data['results'][0]['id']
        
        # Build the movie details query URL
        movie_query_url = f"{tmdb_movie_url}{movie_id}?api_key={tmdb_api_key}"
        movie_response = requests.get(movie_query_url)
        movie_data = movie_response.json()
        
        genres = [genre['name'] for genre in movie_data.get('genres', [])]
        spoken_languages = [lang['english_name'] for lang in movie_data.get('spoken_languages', [])]
        production_countries = [country['name'] for country in movie_data.get('production_countries', [])]
        
        movie_info = {
            'title': title,
            'original_title': movie_data.get('original_title', ''),
            'budget': movie_data.get('budget', ''),
            'original_language': movie_data.get('original_language', ''),
            'homepage': movie_data.get('homepage', ''),
            'overview': movie_data.get('overview', ''),
            'popularity': movie_data.get('popularity', ''),
            'runtime': movie_data.get('runtime', ''),
            'revenue': movie_data.get('revenue', ''),
            'release_date': movie_data.get('release_date', ''),
            'vote_average': movie_data.get('vote_average', ''),
            'vote_count': movie_data.get('vote_count', ''),
            'genres': ", ".join(genres),
            'spoken_languages': ", ".join(spoken_languages),
            'production_countries': ", ".join(production_countries)
        }
        tmdb_movies_list.append(movie_info)
        print(f"Successfully retrieved details for title: {title}")
    except Exception as e:
        print(f"Failed to retrieve details for title: {title}")
        print(f"Error: {e}")


Successfully retrieved details for title: The Attachment Diaries
No movie found for title: What’s Love Got to Do With It?’ Probably a Lo
Successfully retrieved details for title: You Can Live Forever
Successfully retrieved details for title: A Tourist’s Guide to Love
Successfully retrieved details for title: Other People’s Children
Successfully retrieved details for title: One True Loves
Successfully retrieved details for title: The Lost Weekend: A Love Story
Successfully retrieved details for title: A Thousand and One
Successfully retrieved details for title: Your Place or Mine
Successfully retrieved details for title: Love in the Time of Fentanyl
Successfully retrieved details for title: Pamela, a Love Story
Successfully retrieved details for title: In From the Side
Successfully retrieved details for title: After Love
Successfully retrieved details for title: Alcarràs
Successfully retrieved details for title: Nelly & Nadine
Successfully retrieved details for title: Lady Chatterley’s 

In [96]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

In [97]:
#Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print("Preview of TMDb Data:")
print(json.dumps(tmdb_movies_list[:5], indent=4))

Preview of TMDb Data:
[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 2.505,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": "Drama, Mystery, Thriller, Horror",
        "spoken_languages": "Spanish",
        "production_countries": "Argentina"
    },
    {
        "title": "You Can Live Forever",
        "original_title": "You Can Live

In [98]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

In [99]:
### Merge and Clean the Data for Export

In [100]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_df, on='title', how='inner')

# Print the head of the merged DataFrame to check the result
print("Preview of Merged Data:")
print(merged_df.head())

# Verify the merge by checking for the presence of key columns
print("Columns in the merged DataFrame:")
print(merged_df.columns)

# Check for unexpected NaN values
print("Number of NaN values in each column:")
print(merged_df.isnull().sum())

Preview of Merged Data:
                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/you-...   
2  https://www.nytimes.com/2023/04/21/movies/a-to...   
3  https://www.nytimes.com/2023/04/20/movies/othe...   
4  https://www.nytimes.com/2023/04/13/movies/one-...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Religion comes between two girls falling in lo...  The New York Times   
2  Rachael Leigh Cook stars in this bland rom-com...  The New York Times   
3  A radiant Virginie Efira stars as a Parisian t...  The New York Times   
4  A film adaptation of Taylor Jenkins Reid’s nov...  The New York Times   

                                            keywords  \
0  [{'name': 'subject', 'value': 'Movies', 'rank'...   
1  [{'name': 'subject', 'value': 'Movies', 'rank'...   
2  [{'name': '

In [137]:
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Ensure the column contains string representations of lists
    merged_df[column] = merged_df[column].astype(str)
    
    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, "", regex=False)

# Display the fixed DataFrame
print("Fixed Merged DataFrame:")
print(merged_df.head())

Fixed Merged DataFrame:
                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/you-...   
2  https://www.nytimes.com/2023/04/21/movies/a-to...   
3  https://www.nytimes.com/2023/04/20/movies/othe...   
4  https://www.nytimes.com/2023/04/13/movies/one-...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Religion comes between two girls falling in lo...  The New York Times   
2  Rachael Leigh Cook stars in this bland rom-com...  The New York Times   
3  A radiant Virginie Efira stars as a Parisian t...  The New York Times   
4  A film adaptation of Taylor Jenkins Reid’s nov...  The New York Times   

                                            keywords  \
0  [{'name': 'subject', 'value': 'Movies', 'rank'...   
1  [{'name': 'subject', 'value': 'Movies', 'rank'...   
2  [{'name': '

In [175]:
# Drop the "byline.person" column if it exists
columns_to_drop = [col for col in merged_df.columns if 'byline.person' in col]
if columns_to_drop:
    merged_df.drop(columns=columns_to_drop, inplace=True)

In [176]:
print("Columns in the merged DataFrame after attempting to drop specified columns:")
print(merged_df.columns)

Columns in the merged DataFrame after attempting to drop specified columns:
Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'title', 'original_title', 'budget', 'original_language', 'homepage',
       'overview', 'popularity', 'runtime', 'revenue', 'release_date',
       'vote_average', 'vote_count', 'genres', 'spoken_languages',
       'production_countries'],
      dtype='object')


In [177]:
 # Remove duplicate rows and reset the index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop=True, inplace=True)

In [178]:
# Export data to CSV without the index
merged_df.to_csv('merged_cleaned_data.csv', index=False)